# KNN

= 특별한 예측 모델 없이 가장 가까운 데이터 포인트를 기반으로 예측을 수행하는 방법이다.

- 분류와 회귀를 모두 지원한다.

> ***parameter***
>- ***n_neighbor*** *: 홀수로 지정하는것이 좋다.*

In [41]:
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing

In [3]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.datasets import load_boston, fetch_california_housing
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline

## KNeighborsClassifier

- 입력 데이터 포인트와 가장 가까운 k개의 훈련 데이터 포인트가 출력된다.
- k개의 데이토 포인트 중 가장 많은 클래스가 예측결과에 해당한다.

In [12]:
iris = load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df["class"] = iris.target

iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [26]:
X, y = load_iris(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [27]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [52]:
knc = KNeighborsClassifier().fit(X_train, y_train)

print("train score : {:.3f}".format(knc.score(X_train, y_train)))
print("test score : {:.3f}".format(knc.score(X_test, y_test)))

train score : 0.983
test score : 0.933


In [54]:
knc_scaler = KNeighborsClassifier().fit(X_train_scale, y_train)

print("scale train score : {:.3f}".format(knc_scaler.score(X_train_scale, y_train)))
print("scale test score : {:.3f}".format(knc_scaler.score(X_test_scale, y_test)))

scale train score : 0.975
scale test score : 0.933


In [32]:
cross_validate(estimator = KNeighborsClassifier(),
              X = X,
              y = y,
              cv =5,
              n_jobs = multiprocessing.cpu_count(),
              verbose = True)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    2.3s finished


{'fit_time': array([0.00199485, 0.00199485, 0.00199485, 0.00199485, 0.00099492]),
 'score_time': array([0.00299096, 0.00299096, 0.00299096, 0.00299096, 0.00299168]),
 'test_score': array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ])}

In [34]:
param_grid = [{"n_neighbors" : [1, 3, 5, 7, 9],
              "weights" : ["uniform", "distance"],
              "algorithm" : ["ball_tree", "kd_tree", "brute"]}]

In [35]:
gs = GridSearchCV(estimator = KNeighborsClassifier(),
                 param_grid = param_grid,
                 n_jobs = multiprocessing.cpu_count(),
                 verbose = True)

gs.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid=[{'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                          'n_neighbors': [1, 3, 5, 7, 9],
                          'weights': ['uniform', 'distance']}],
             verbose=True)

In [36]:
gs.best_estimator_ # ball_tree, 7개가 최적의 파라미터

KNeighborsClassifier(algorithm='ball_tree', n_neighbors=7)

In [37]:
print("best score : {:.3f}".format(gs.best_score_))

best score : 0.980


## KNeighborsRegressor

- KNeighborsClassifier와 마찬가지로 예측에 이웃 데이터 포인트를 사용한다.
- 이웃 데이터 포인트의 평균이 예측결과가 된다.

In [44]:
boston = load_boston()

boston_df = pd.DataFrame(data = boston.data, columns = boston.feature_names)
boston_df["target"] = boston.target
boston_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [55]:
X, y = boston.data, boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [56]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [57]:
knr = KNeighborsRegressor().fit(X_train, y_train)

print("train score : {:.3f}".format(knr.score(X_train, y_train)))
print("test score : {:.3f}".format(knr.score(X_test, y_test))) # test에서의 성능이 낮음

train score : 0.708
test score : 0.564


In [58]:
knr_scaler = KNeighborsRegressor().fit(X_train_scale, y_train)

print("scale train score : {:.3f}".format(knr_scaler.score(X_train_scale, y_train)))
print("scale test score : {:.3f}".format(knr_scaler.score(X_test_scale, y_test))) # 스케일링 이후 성능이 향상되었음

scale train score : 0.825
scale test score : 0.747
